# KEN3450 COMPETITION


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, LassoLarsIC
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

## Read the data
- drop the id column for training data, but remember it for the test data
- extract the label (house price) and transform it to log space
- join the training and testing data into a single dataframe for easier transformation with pandas


In [2]:
# read the data 
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# get the training and testing data
x_train_raw = train_data.drop(['shares'],axis=1)
x_test_raw = test_data

# remember the lenghts so we can split the data later
train_length = len(train_data)
test_length = len(test_data)

# concatenate train + test data into one df
all_data = pd.concat([x_train_raw, x_test_raw])

# get the training data - house prices
# normal prices have a skew so adjust them to log space
y_train = train_data['shares']


#display the data
pd.set_option('display.max_columns', 500)
x_test_raw.head(2)


,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,topic_01,topic_02,topic_03,topic_04,topic_05,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,title_subjectivity,title_sentiment_polarity
0,9,531,0.503788,1.0,0.665635,9,0,1,0,4.404896,7,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,1,0,0,0,0,0,0,0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.000000,0.000000
1,10,370,0.559889,1.0,0.698198,2,2,0,0,4.359459,9,0,0,0,0,1,0,0.0,0.0,0.0,8500.0,8500,8500.0,1,0,0,0,0,0,0,0,0.022245,0.306718,0.022231,0.022224,0.626582,0.437409,0.071184,0.029730,0.027027,0.523810,0.476190,0.642857,0.214286


### Transforming features
- missing -> replace with median 
- categorical -> One hot encoding
- numerical -> use Robust scaler that is robust to outliers


In [3]:
# a nice trick to find out numeric vs categorical features
numerical_features = x_train_raw.columns[x_train_raw.dtypes != 'object']
categorical_features = x_train_raw.columns[x_train_raw.dtypes == 'object']

# encode missing numbers as a special large number
all_data[numerical_features] = all_data[numerical_features].fillna(0)

# encode missing data as a special category -> missing
all_data[categorical_features] = all_data[categorical_features].fillna("Missing")

all_data.head(2)

# transform numeric variables 
ss = RobustScaler() # StandardScaler()
#all_data[numerical_features] = ss.fit_transform(all_data[numerical_features])

# transform categorical variables
all_data = pd.get_dummies(data=all_data, columns=categorical_features)

### Re-split the data back to original proportions¶


In [4]:
# re-split again
x_train = all_data.head(train_length)
x_test = all_data.tail(test_length)

print("Raw features vs features in transformed model:", x_train_raw.shape[1] ,'vs', x_train.shape[1], 'features')
print("Length train vs test:" , train_length , ' vs ', test_length)

Raw features vs features in transformed model: 44 vs 44 features
Length train vs test: 29733  vs  9911


## XGBoost regressor


In [5]:
Xgbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01, max_depth=7, 
                                max_features='sqrt',min_samples_leaf=15, min_samples_split=10, 
                                loss='ls',random_state = 13)
# est.fit(x_train, y_train)
Xgbr2 = GradientBoostingRegressor(n_estimators=3000, learning_rate=1, max_depth=18, 
                                max_features='sqrt',min_samples_leaf=15, min_samples_split=10, 
                                loss='ls',random_state = 23)

# est.fit(x_train, y_train)
Xgbr3 = GradientBoostingRegressor(n_estimators=2500, learning_rate=0.1, max_depth=15, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=5, 
                                loss='ls',random_state = 33)

Xgbr4 = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.01, max_depth=10, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=5, 
                                loss='ls',random_state = 33)

Xgbr5 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.001, max_depth=3, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=35, 
                                loss='ls',random_state = 44)

Xgbr6 = GradientBoostingRegressor(n_estimators=500, learning_rate=0.001, max_depth=3, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=25, 
                                loss='ls',random_state = 44)

Xgbr7 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.001, max_depth=10, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=55, 
                                loss='ls',random_state = 44)

Xgbr8 = GradientBoostingRegressor(n_estimators=50, learning_rate=0.001, max_depth=20, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=35, 
                                loss='ls',random_state = 44)

Xgbr9 = GradientBoostingRegressor(n_estimators=10, learning_rate=0.001, max_depth=15, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=20, 
                                loss='ls',random_state = 44)

Xgbr10 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.001, max_depth=5, 
                                max_features='sqrt',min_samples_leaf=5, min_samples_split=10, 
                                loss='ls',random_state = 44)

Xgbr11 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01, max_depth=7, 
                                max_features='auto',min_samples_leaf=15, min_samples_split=10, 
                                loss='ls',random_state = 13)
Xgbr12 = GradientBoostingRegressor(n_estimators=3000, learning_rate=1, max_depth=18, 
                                max_features='log2',min_samples_leaf=15, min_samples_split=10, 
                                loss='ls',random_state = 23)

Xgbr13 = GradientBoostingRegressor(n_estimators=3500, learning_rate=0.5, max_depth=15, 
                                max_features='log2',min_samples_leaf=5, min_samples_split=5, 
                                loss='ls',random_state = 33)

Xgbr14 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=10, 
                                max_features='auto',min_samples_leaf=5, min_samples_split=5, 
                                loss='ls',random_state = 99)

Xgbr15 = GradientBoostingRegressor(n_estimators=20, learning_rate=0.01, max_depth=3, 
                                max_features='log2',min_samples_leaf=5, min_samples_split=35, 
                                loss='ls',random_state = 44)

Xgbr16 = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=3, 
                                max_features='auto',min_samples_leaf=5, min_samples_split=25, 
                                loss='ls',random_state = 44)

Xgbr17 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, max_depth=10, 
                                max_features='log2',min_samples_leaf=5, min_samples_split=55, 
                                loss='ls',random_state = 88)

Xgbr18 = GradientBoostingRegressor(n_estimators=250, learning_rate=0.5, max_depth=20, 
                                max_features='log2',min_samples_leaf=5, min_samples_split=35, 
                                loss='ls',random_state = 77)

Xgbr19 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, max_depth=15, 
                                max_features='log2',min_samples_leaf=5, min_samples_split=20, 
                                loss='ls',random_state = 66)

Xgbr20 = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, 
                                max_features='log2',min_samples_leaf=5, min_samples_split=10, 
                                loss='ls',random_state = 55)


from sklearn import linear_model

regr = linear_model.LinearRegression(normalize=False)
regr2 = linear_model.LinearRegression(normalize=True)
regr3 = linear_model.LinearRegression(normalize=True)

lasso =  Lasso(alpha =0.0005)
lasso2 =  Lasso(alpha =0.005)
lasso3 =  Lasso(alpha =0.05) 
lasso4 =  Lasso(alpha =0.5) 
lasso5 =  Lasso(alpha = 0.9) 

ridge = Ridge(alpha =0.0005)
ridge2 = Ridge(alpha =0.005)
ridge3 = Ridge(alpha =0.05)
ridge4 = Ridge(alpha =0.5)
ridge5 = Ridge(alpha =0.9)

ENet1 =  ElasticNet(alpha=0.0005)
ENet2 =  ElasticNet(alpha=0.005)
ENet3 =  ElasticNet(alpha=0.05)
ENet4 =  ElasticNet(alpha=0.5)
ENet5 =  ElasticNet(alpha=0.9)

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

averaged_models = AveragingModels(models = (Xgbr,Xgbr2,Xgbr3,Xgbr4,Xgbr5,Xgbr6,Xgbr7,Xgbr8,Xgbr9,Xgbr10,
                                            Xgbr11,Xgbr12,Xgbr13,Xgbr14,Xgbr15,Xgbr16,Xgbr17,Xgbr18,Xgbr19,Xgbr20,
                                            regr,regr2,regr3,lasso,lasso2,lasso3,lasso4,lasso5,
                                            ridge,ridge2,ridge3,ridge4,ridge5,ENet1,ENet2,ENet3,ENet4,ENet5))


In [6]:
best_model = averaged_models.fit(x_train, y_train)

/Volumes/SD/Applications/miniconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/Volumes/SD/Applications/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


### Write the submission

In [18]:
y_test = best_model.predict(x_test)
result = pd.DataFrame({'shares': y_test.ravel()})
result['id'] = result.index + 1

result = result[['id','shares']]
result[result <0] = y_train.mean() / 4
result.to_csv('submission.csv',index=False)


In [19]:
result[result['shares'] < 0]

,id,shares
